# PARTE 1: ANÁLISIS ÉTICO Y DE FAIRNESS

## Entrega 4 - ConcentraciónAI

Este notebook implementa un análisis completo de fairness y consideraciones éticas para los modelos entrenados en la Entrega 3 utilizando el dataset del Titanic.

### Objetivos:
1. **Evaluación de Sesgos Algorítmicos**: Implementar métricas cuantitativas de fairness
2. **Análisis Interseccional**: Examinar disparidades entre múltiples grupos protegidos
3. **Simulación de Decisiones**: Evaluar el impacto ético en escenarios reales
4. **Reflexión Ética**: Análisis profundo de dilemas éticos y responsabilidad

### Contexto Ético
El dataset del Titanic representa un evento histórico real donde las decisiones de vida o muerte estuvieron influenciadas por factores socioeconómicos, género y edad. Al construir modelos predictivos sobre estos datos, debemos ser conscientes de que:
- Los patrones reflejan desigualdades históricas
- Los modelos pueden perpetuar o amplificar estos sesgos
- Las decisiones algorítmicas tienen implicaciones éticas profundas

In [1]:
# Importar librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

# Librerías para visualización
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Librerías de machine learning
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    brier_score_loss
)
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import LabelEncoder

# Intentar importar librerías de fairness (pueden no estar instaladas)
try:
    from fairlearn.metrics import (
        demographic_parity_ratio,
        demographic_parity_difference,
        equalized_odds_ratio,
        equalized_odds_difference,
        MetricFrame
    )
    fairlearn_available = True
    print("✓ Fairlearn disponible")
except ImportError:
    fairlearn_available = False
    print("⚠️ Fairlearn no disponible - implementaremos métricas manualmente")

try:
    from aif360.metrics import BinaryLabelDatasetMetric
    from aif360.algorithms.preprocessing import Reweighing
    from aif360.algorithms.postprocessing import EqOddsPostprocessing
    aif360_available = True
    print("✓ AIF360 disponible")
except ImportError:
    aif360_available = False
    print("⚠️ AIF360 no disponible - implementaremos métricas manualmente")

print("Librerías importadas exitosamente")

✓ Fairlearn disponible
⚠️ AIF360 no disponible - implementaremos métricas manualmente
Librerías importadas exitosamente


In [2]:
# Cargar el dataset procesado de la Entrega 2
print("Cargando datos...")
df = pd.read_csv('../Entrega2/data/Titanic_Dataset_Featured.csv')

# Verificar la estructura del dataset
print(f"Forma del dataset: {df.shape}")
print("\nColumnas disponibles:")
print(df.columns.tolist())

# Mostrar información básica sobre grupos protegidos
print("\n=== ANÁLISIS DEMOGRÁFICO ===")
print(f"Distribución por Sexo:")
print(df['Sex'].value_counts())
print(f"\nDistribución por Clase:")
print(df['Pclass'].value_counts())
print(f"\nDistribución por Edad (grupos):")
df['Age_Group'] = pd.cut(df['Age'], bins=[0, 16, 60, 100], labels=['Child', 'Adult', 'Elder'])
print(df['Age_Group'].value_counts())

# Tasa de supervivencia general
survival_rate = df['Survived'].mean()
print(f"\nTasa de supervivencia general: {survival_rate:.3f} ({survival_rate*100:.1f}%)")

# Mostrar primeras filas
print("\nPrimeras 5 filas del dataset:")
df.head()

Cargando datos...
Forma del dataset: (891, 36)

Columnas disponibles:
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Has_Cabin', 'Age_Group', 'Title', 'FamilySize', 'IsAlone', 'AgeGroup', 'FarePerPerson', 'FarePerPerson_Quintile', 'TicketFrequency', 'TicketFreq_Category', 'CabinDeck', 'Mother', 'NameLength', 'NameLength_Category', 'NameLength_Quintile', 'TicketPrefix', 'TicketPrefix_Category', 'Fare_log', 'FarePerPerson_log', 'Age_sqrt', 'NameLength_sqrt', 'FamilySize_Category', 'IsMinor', 'DeckCategory']

=== ANÁLISIS DEMOGRÁFICO ===
Distribución por Sexo:
Sex
male      577
female    314
Name: count, dtype: int64

Distribución por Clase:
Pclass
3    491
1    216
2    184
Name: count, dtype: int64

Distribución por Edad (grupos):
Age_Group
Adult    769
Child    100
Elder     22
Name: count, dtype: int64

Tasa de supervivencia general: 0.384 (38.4%)

Primeras 5 filas del dataset:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,NameLength_Quintile,TicketPrefix,TicketPrefix_Category,Fare_log,FarePerPerson_log,Age_sqrt,NameLength_sqrt,FamilySize_Category,IsMinor,DeckCategory
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,Q2,A_,Other,2.110213,2.110213,4.690416,4.795832,Pequeña,0,Inferior
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,Q5,PC,PC,4.280593,4.280593,6.164414,7.141428,Pequeña,0,Superior
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,Q2,STON_O,Other,2.188856,2.188856,5.099020,4.690416,Solo,0,Inferior
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,Q5,Numeric,Numeric,3.990834,3.316003,5.916080,6.633250,Pequeña,0,Superior
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,Q3,Numeric,Numeric,2.202765,2.202765,5.916080,4.898979,Solo,0,Inferior


In [3]:
# Configurar el notebook para trabajar correctamente
import os
import sys
sys.path.append('..')

# Configurar estilo de plots
plt.style.use('default')
sns.set_palette("husl")

# Configurar plotly para mejor visualización
import plotly.io as pio
pio.templates.default = "plotly_white"

print("Notebook configurado exitosamente")

Notebook configurado exitosamente


In [4]:
# Cargar los modelos entrenados de la Entrega 3
models_path = '../Entrega3/models/'

# Diccionario para almacenar los modelos
models = {}
model_results = {}

# Lista de modelos disponibles
model_files = {
    'RandomForest': 'randomforest_model.pkl',
    'LogisticRegression': 'logisticregression_model.pkl', 
    'SVM': 'svm_model.pkl',
    'XGBoost': 'xgboost_model.pkl'
}

# Cargar cada modelo
print("Cargando modelos entrenados...")
for name, filename in model_files.items():
    try:
        model_path = os.path.join(models_path, filename)
        with open(model_path, 'rb') as f:
            models[name] = pickle.load(f)
        print(f"✓ {name} cargado exitosamente")
        
        # Cargar resultados si están disponibles
        results_file = filename.replace('_model.pkl', '_results.pkl')
        results_path = os.path.join(models_path, results_file)
        if os.path.exists(results_path):
            with open(results_path, 'rb') as f:
                model_results[name] = pickle.load(f)
                
    except Exception as e:
        print(f"❌ Error cargando {name}: {e}")

print(f"\nModelos cargados: {list(models.keys())}")
print(f"Resultados disponibles: {list(model_results.keys())}")

Cargando modelos entrenados...
❌ Error cargando RandomForest: invalid load key, '\x0d'.
❌ Error cargando LogisticRegression: invalid load key, '\x0b'.
❌ Error cargando SVM: invalid load key, '\x0b'.
❌ Error cargando RandomForest: invalid load key, '\x0d'.
❌ Error cargando LogisticRegression: invalid load key, '\x0b'.
❌ Error cargando SVM: invalid load key, '\x0b'.
✓ XGBoost cargado exitosamente
❌ Error cargando XGBoost: invalid load key, '\x03'.

Modelos cargados: ['XGBoost']
Resultados disponibles: []
✓ XGBoost cargado exitosamente
❌ Error cargando XGBoost: invalid load key, '\x03'.

Modelos cargados: ['XGBoost']
Resultados disponibles: []


In [5]:
# Preparar los datos para el análisis de fairness
# Seleccionar las características más importantes (basado en análisis previos)

# Características numéricas principales
numeric_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson']

# Codificar características categóricas
df_fairness = df.copy()

# Codificar sexo (0: female, 1: male)
df_fairness['Sex_encoded'] = (df_fairness['Sex'] == 'male').astype(int)

# Codificar embarque
embarked_encoder = LabelEncoder()
df_fairness['Embarked_encoded'] = embarked_encoder.fit_transform(df_fairness['Embarked'].fillna('S'))

# Codificar título
title_encoder = LabelEncoder()  
df_fairness['Title_encoded'] = title_encoder.fit_transform(df_fairness['Title'])

# Crear características finales para el modelo
feature_columns = numeric_features + ['Sex_encoded', 'Embarked_encoded', 'Title_encoded', 'Has_Cabin', 'IsAlone']

# Preparar X e y
X = df_fairness[feature_columns].fillna(0)
y = df_fairness['Survived']

print("Preparación de datos completada")
print(f"Características utilizadas: {feature_columns}")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

# Definir grupos protegidos para análisis de fairness
protected_attrs = {
    'Gender': df_fairness['Sex'],
    'Class': df_fairness['Pclass'],
    'Age_Group': df_fairness['Age_Group']
}

print(f"\nGrupos protegidos definidos: {list(protected_attrs.keys())}")

Preparación de datos completada
Características utilizadas: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson', 'Sex_encoded', 'Embarked_encoded', 'Title_encoded', 'Has_Cabin', 'IsAlone']
Forma de X: (891, 12)
Forma de y: (891,)

Grupos protegidos definidos: ['Gender', 'Class', 'Age_Group']


In [6]:
# Entrenar un modelo Random Forest para análisis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Obtener predicciones y probabilidades
rf_predictions = rf_model.predict(X_test)
rf_probabilities = rf_model.predict_proba(X_test)[:, 1]

# Evaluación básica
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, rf_predictions)
print(f"Accuracy del Random Forest: {accuracy:.3f}")
print(f"Forma de los datos de prueba: {X_test.shape}")

# Guardar el modelo entrenado en nuestro diccionario
models['RandomForest_Current'] = rf_model

# Crear DataFrame con datos de prueba y resultados
test_indices = X_test.index
df_test = df_fairness.loc[test_indices].copy()
df_test['Predicted'] = rf_predictions
df_test['Probability'] = rf_probabilities

print(f"\nDataset de prueba preparado con {len(df_test)} muestras")
print("Distribución de predicciones:")
print(df_test['Predicted'].value_counts())

Accuracy del Random Forest: 0.788
Forma de los datos de prueba: (179, 12)

Dataset de prueba preparado con 179 muestras
Distribución de predicciones:
Predicted
0    112
1     67
Name: count, dtype: int64


In [7]:
class FairnessAnalyzer:
    """
    Clase para realizar análisis completo de fairness en modelos de ML
    Implementa métricas de disparidad demográfica, igualdad de oportunidad,
    odds equalizados y calibración por grupo.
    """
    
    def __init__(self, df, true_col='Survived', pred_col='Predicted', prob_col='Probability'):
        self.df = df.copy()
        self.true_col = true_col
        self.pred_col = pred_col
        self.prob_col = prob_col
        
    def demographic_parity(self, group_col):
        """
        Calcula disparidad demográfica (Statistical Parity)
        P(Ŷ=1|G=0) ≈ P(Ŷ=1|G=1)
        """
        results = {}
        groups = self.df[group_col].unique()
        
        for group in groups:
            group_mask = self.df[group_col] == group
            group_data = self.df[group_mask]
            
            pred_rate = group_data[self.pred_col].mean()
            total_group = len(group_data)
            
            results[group] = {
                'positive_prediction_rate': pred_rate,
                'count': total_group
            }
        
        # Calcular ratios y diferencias
        groups_list = list(groups)
        if len(groups_list) >= 2:
            group1, group2 = groups_list[0], groups_list[1]
            rate1 = results[group1]['positive_prediction_rate']
            rate2 = results[group2]['positive_prediction_rate']
            
            results['demographic_parity_ratio'] = min(rate1, rate2) / max(rate1, rate2)
            results['demographic_parity_difference'] = abs(rate1 - rate2)
        
        return results
    
    def equal_opportunity(self, group_col):
        """
        Calcula igualdad de oportunidad (Equal Opportunity)
        P(Ŷ=1|Y=1,G=0) ≈ P(Ŷ=1|Y=1,G=1)
        True Positive Rate por grupo
        """
        results = {}
        groups = self.df[group_col].unique()
        
        for group in groups:
            group_mask = self.df[group_col] == group
            positive_actual = self.df[group_mask & (self.df[self.true_col] == 1)]
            
            if len(positive_actual) > 0:
                tpr = positive_actual[self.pred_col].mean()
                total_positives = len(positive_actual)
                true_positives = positive_actual[self.pred_col].sum()
            else:
                tpr = 0
                total_positives = 0
                true_positives = 0
            
            results[group] = {
                'true_positive_rate': tpr,
                'true_positives': true_positives,
                'total_actual_positives': total_positives
            }
        
        # Calcular diferencias
        groups_list = list(groups)
        if len(groups_list) >= 2:
            group1, group2 = groups_list[0], groups_list[1]
            tpr1 = results[group1]['true_positive_rate']
            tpr2 = results[group2]['true_positive_rate']
            
            if max(tpr1, tpr2) > 0:
                results['equal_opportunity_ratio'] = min(tpr1, tpr2) / max(tpr1, tpr2)
            else:
                results['equal_opportunity_ratio'] = 1.0
            results['equal_opportunity_difference'] = abs(tpr1 - tpr2)
        
        return results
    
    def equalized_odds(self, group_col):
        """
        Calcula odds equalizados (Equalized Odds)
        TPR y FPR similares entre grupos
        """
        results = {}
        groups = self.df[group_col].unique()
        
        for group in groups:
            group_mask = self.df[group_col] == group
            group_data = self.df[group_mask]
            
            # True Positive Rate
            actual_positives = group_data[group_data[self.true_col] == 1]
            if len(actual_positives) > 0:
                tpr = actual_positives[self.pred_col].mean()
            else:
                tpr = 0
            
            # False Positive Rate
            actual_negatives = group_data[group_data[self.true_col] == 0]
            if len(actual_negatives) > 0:
                fpr = actual_negatives[self.pred_col].mean()
            else:
                fpr = 0
            
            results[group] = {
                'true_positive_rate': tpr,
                'false_positive_rate': fpr,
                'actual_positives': len(actual_positives),
                'actual_negatives': len(actual_negatives)
            }
        
        # Calcular diferencias
        groups_list = list(groups)
        if len(groups_list) >= 2:
            group1, group2 = groups_list[0], groups_list[1]
            tpr_diff = abs(results[group1]['true_positive_rate'] - results[group2]['true_positive_rate'])
            fpr_diff = abs(results[group1]['false_positive_rate'] - results[group2]['false_positive_rate'])
            
            results['tpr_difference'] = tpr_diff
            results['fpr_difference'] = fpr_diff
            results['equalized_odds_satisfied'] = (tpr_diff < 0.1) and (fpr_diff < 0.1)
        
        return results
    
    def calibration_by_group(self, group_col, n_bins=10):
        """
        Analiza calibración por grupo
        P(Y=1|Ŷ=p,G=g) ≈ p para todo g
        """
        results = {}
        groups = self.df[group_col].unique()
        
        for group in groups:
            group_mask = self.df[group_col] == group
            group_data = self.df[group_mask]
            
            if len(group_data) > 0:
                y_true = group_data[self.true_col]
                y_prob = group_data[self.prob_col]
                
                # Usar calibration_curve de sklearn
                fraction_of_positives, mean_predicted_value = calibration_curve(
                    y_true, y_prob, n_bins=min(n_bins, len(group_data)//2)
                )
                
                # Calcular Brier Score
                brier_score = brier_score_loss(y_true, y_prob)
                
                results[group] = {
                    'fraction_of_positives': fraction_of_positives,
                    'mean_predicted_value': mean_predicted_value,
                    'brier_score': brier_score,
                    'count': len(group_data)
                }
        
        return results
    
    def comprehensive_analysis(self, group_col):
        """
        Realiza análisis completo de fairness para un grupo
        """
        print(f"\n{'='*60}")
        print(f"ANÁLISIS DE FAIRNESS: {group_col.upper()}")
        print(f"{'='*60}")
        
        # 1. Disparidad Demográfica
        dp_results = self.demographic_parity(group_col)
        print(f"\n1. DISPARIDAD DEMOGRÁFICA (Statistical Parity)")
        print("-" * 50)
        for group in self.df[group_col].unique():
            rate = dp_results[group]['positive_prediction_rate']
            count = dp_results[group]['count']
            print(f"{group}: {rate:.3f} ({rate*100:.1f}%) - N={count}")
        
        if 'demographic_parity_ratio' in dp_results:
            ratio = dp_results['demographic_parity_ratio']
            diff = dp_results['demographic_parity_difference']
            print(f"Ratio: {ratio:.3f} | Diferencia: {diff:.3f}")
            if ratio < 0.8:
                print("⚠️  SESGO DETECTADO: Disparidad demográfica significativa")
            else:
                print("✓ Disparidad demográfica aceptable")
        
        # 2. Igualdad de Oportunidad
        eo_results = self.equal_opportunity(group_col)
        print(f"\n2. IGUALDAD DE OPORTUNIDAD (Equal Opportunity)")
        print("-" * 50)
        for group in self.df[group_col].unique():
            tpr = eo_results[group]['true_positive_rate']
            tp = eo_results[group]['true_positives']
            total = eo_results[group]['total_actual_positives']
            print(f"{group}: TPR={tpr:.3f} ({tp}/{total})")
        
        if 'equal_opportunity_ratio' in eo_results:
            ratio = eo_results['equal_opportunity_ratio']
            diff = eo_results['equal_opportunity_difference']
            print(f"Ratio: {ratio:.3f} | Diferencia: {diff:.3f}")
            if ratio < 0.8:
                print("⚠️  SESGO DETECTADO: Desigualdad de oportunidad significativa")
            else:
                print("✓ Igualdad de oportunidad aceptable")
        
        # 3. Odds Equalizados
        eq_results = self.equalized_odds(group_col)
        print(f"\n3. ODDS EQUALIZADOS (Equalized Odds)")
        print("-" * 50)
        for group in self.df[group_col].unique():
            tpr = eq_results[group]['true_positive_rate']
            fpr = eq_results[group]['false_positive_rate']
            print(f"{group}: TPR={tpr:.3f}, FPR={fpr:.3f}")
        
        if 'equalized_odds_satisfied' in eq_results:
            satisfied = eq_results['equalized_odds_satisfied']
            tpr_diff = eq_results['tpr_difference']
            fpr_diff = eq_results['fpr_difference']
            print(f"TPR diff: {tpr_diff:.3f} | FPR diff: {fpr_diff:.3f}")
            if satisfied:
                print("✓ Odds equalizados satisfechos")
            else:
                print("⚠️  SESGO DETECTADO: Odds no equalizados")
        
        # 4. Calibración por Grupo
        cal_results = self.calibration_by_group(group_col)
        print(f"\n4. CALIBRACIÓN POR GRUPO")
        print("-" * 50)
        for group in self.df[group_col].unique():
            brier = cal_results[group]['brier_score']
            count = cal_results[group]['count']
            print(f"{group}: Brier Score={brier:.3f} (N={count})")
        
        return {
            'demographic_parity': dp_results,
            'equal_opportunity': eo_results,
            'equalized_odds': eq_results,
            'calibration': cal_results
        }

print("Clase FairnessAnalyzer implementada exitosamente")

Clase FairnessAnalyzer implementada exitosamente


## 1.1 Evaluación de Sesgos Algorítmicos

### A. Análisis Cuantitativo de Fairness

En esta sección implementamos y analizamos las siguientes métricas de fairness para cada grupo protegido (género, clase, edad):

1. **DISPARIDAD DEMOGRÁFICA (Statistical Parity)**: P(Ŷ=1|G=0) ≈ P(Ŷ=1|G=1)
2. **IGUALDAD DE OPORTUNIDAD (Equal Opportunity)**: P(Ŷ=1|Y=1,G=0) ≈ P(Ŷ=1|Y=1,G=1)
3. **ODDS EQUALIZADOS (Equalized Odds)**: TPR y FPR similares entre grupos
4. **CALIBRACIÓN POR GRUPO**: P(Y=1|Ŷ=p,G=g) ≈ p para todo g

Las métricas se evalúan usando umbrales estándar:
- **Ratio < 0.8**: Indica sesgo significativo
- **Diferencia > 0.1**: Sugiere disparidad notable
- **Brier Score**: Menor es mejor para calibración

In [8]:
# Crear instancia del analizador de fairness
analyzer = FairnessAnalyzer(df_test)

# ANÁLISIS POR GÉNERO
print("INICIANDO ANÁLISIS CUANTITATIVO DE FAIRNESS")
print("=" * 80)

# Realizar análisis completo por género
gender_results = analyzer.comprehensive_analysis('Sex')

INICIANDO ANÁLISIS CUANTITATIVO DE FAIRNESS

ANÁLISIS DE FAIRNESS: SEX

1. DISPARIDAD DEMOGRÁFICA (Statistical Parity)
--------------------------------------------------
male: 0.186 (18.6%) - N=118
female: 0.738 (73.8%) - N=61
Ratio: 0.253 | Diferencia: 0.551
⚠️  SESGO DETECTADO: Disparidad demográfica significativa

2. IGUALDAD DE OPORTUNIDAD (Equal Opportunity)
--------------------------------------------------
male: TPR=0.417 (10/24)
female: TPR=0.867 (39/45)
Ratio: 0.481 | Diferencia: 0.450
⚠️  SESGO DETECTADO: Desigualdad de oportunidad significativa

3. ODDS EQUALIZADOS (Equalized Odds)
--------------------------------------------------
male: TPR=0.417, FPR=0.128
female: TPR=0.867, FPR=0.375
TPR diff: 0.450 | FPR diff: 0.247
⚠️  SESGO DETECTADO: Odds no equalizados

4. CALIBRACIÓN POR GRUPO
--------------------------------------------------
male: Brier Score=0.148 (N=118)
female: Brier Score=0.140 (N=61)


In [9]:
# ANÁLISIS POR CLASE SOCIAL
class_results = analyzer.comprehensive_analysis('Pclass')


ANÁLISIS DE FAIRNESS: PCLASS

1. DISPARIDAD DEMOGRÁFICA (Statistical Parity)
--------------------------------------------------
3: 0.180 (18.0%) - N=100
2: 0.559 (55.9%) - N=34
1: 0.667 (66.7%) - N=45
Ratio: 0.322 | Diferencia: 0.379
⚠️  SESGO DETECTADO: Disparidad demográfica significativa

2. IGUALDAD DE OPORTUNIDAD (Equal Opportunity)
--------------------------------------------------
3: TPR=0.500 (12/24)
2: TPR=0.900 (18/20)
1: TPR=0.760 (19/25)
Ratio: 0.556 | Diferencia: 0.400
⚠️  SESGO DETECTADO: Desigualdad de oportunidad significativa

3. ODDS EQUALIZADOS (Equalized Odds)
--------------------------------------------------
3: TPR=0.500, FPR=0.079
2: TPR=0.900, FPR=0.071
1: TPR=0.760, FPR=0.550
TPR diff: 0.400 | FPR diff: 0.008
⚠️  SESGO DETECTADO: Odds no equalizados

4. CALIBRACIÓN POR GRUPO
--------------------------------------------------
3: Brier Score=0.140 (N=100)
2: Brier Score=0.103 (N=34)
1: Brier Score=0.189 (N=45)


In [10]:
# ANÁLISIS POR GRUPOS DE EDAD
age_results = analyzer.comprehensive_analysis('Age_Group')


ANÁLISIS DE FAIRNESS: AGE_GROUP

1. DISPARIDAD DEMOGRÁFICA (Statistical Parity)
--------------------------------------------------
Adult: 0.349 (34.9%) - N=149
Child: 0.542 (54.2%) - N=24
Elder: 0.333 (33.3%) - N=6
Ratio: 0.644 | Diferencia: 0.193
⚠️  SESGO DETECTADO: Disparidad demográfica significativa

2. IGUALDAD DE OPORTUNIDAD (Equal Opportunity)
--------------------------------------------------
Adult: TPR=0.673 (37/55)
Child: TPR=0.917 (11/12)
Elder: TPR=0.500 (1/2)
Ratio: 0.734 | Diferencia: 0.244
⚠️  SESGO DETECTADO: Desigualdad de oportunidad significativa

3. ODDS EQUALIZADOS (Equalized Odds)
--------------------------------------------------
Adult: TPR=0.673, FPR=0.160
Child: TPR=0.917, FPR=0.167
Elder: TPR=0.500, FPR=0.250
TPR diff: 0.244 | FPR diff: 0.007
⚠️  SESGO DETECTADO: Odds no equalizados

4. CALIBRACIÓN POR GRUPO
--------------------------------------------------
Adult: Brier Score=0.151 (N=149)
Child: Brier Score=0.088 (N=24)
Elder: Brier Score=0.233 (N=6)


In [11]:
# Función para crear visualizaciones de fairness
def create_fairness_visualizations(analyzer, df_test):
    """
    Crea visualizaciones completas para el análisis de fairness
    """
    
    # 1. Dashboard de Métricas de Fairness
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=[
            'Disparidad Demográfica por Género',
            'Disparidad Demográfica por Clase', 
            'Disparidad Demográfica por Edad',
            'True Positive Rate por Grupo',
            'False Positive Rate por Grupo',
            'Brier Score (Calibración)'
        ],
        specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]]
    )
    
    # Datos para las visualizaciones
    groups_data = {
        'Sex': ['male', 'female'],
        'Pclass': [3, 2, 1], 
        'Age_Group': ['Adult', 'Child', 'Elder']
    }
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']
    
    # Fila 1: Disparidad Demográfica
    for idx, (group_col, groups) in enumerate(groups_data.items()):
        dp_data = analyzer.demographic_parity(group_col)
        
        rates = [dp_data[group]['positive_prediction_rate'] for group in groups]
        
        fig.add_trace(
            go.Bar(
                x=[str(g) for g in groups],
                y=rates,
                name=f'Pred Rate {group_col}',
                marker_color=colors[idx],
                showlegend=False
            ),
            row=1, col=idx+1
        )
    
    # Fila 2: TPR, FPR y Calibración
    for idx, (group_col, groups) in enumerate(groups_data.items()):
        eo_data = analyzer.equal_opportunity(group_col)
        eq_data = analyzer.equalized_odds(group_col)
        cal_data = analyzer.calibration_by_group(group_col)
        
        if idx == 0:  # TPR
            tprs = [eo_data[group]['true_positive_rate'] for group in groups]
            fig.add_trace(
                go.Bar(
                    x=[str(g) for g in groups],
                    y=tprs,
                    name='TPR',
                    marker_color='#2ECC71',
                    showlegend=False
                ),
                row=2, col=1
            )
        elif idx == 1:  # FPR
            fprs = [eq_data[group]['false_positive_rate'] for group in groups]
            fig.add_trace(
                go.Bar(
                    x=[str(g) for g in groups],
                    y=fprs,
                    name='FPR',
                    marker_color='#E74C3C',
                    showlegend=False
                ),
                row=2, col=2
            )
        else:  # Brier Score
            briers = [cal_data[group]['brier_score'] for group in groups]
            fig.add_trace(
                go.Bar(
                    x=[str(g) for g in groups],
                    y=briers,
                    name='Brier Score',
                    marker_color='#9B59B6',
                    showlegend=False
                ),
                row=2, col=3
            )
    
    fig.update_layout(
        title_text="📊 Dashboard de Métricas de Fairness",
        title_x=0.5,
        height=600,
        font=dict(size=12)
    )
    
    # Agregar líneas de referencia para umbrales
    for i in range(1, 4):
        fig.add_hline(y=0.8, line_dash="dash", line_color="red", 
                     annotation_text="Umbral crítico", row=1, col=i)
    
    fig.show()
    
    # 2. Heatmap de Supervivencia Real vs Predicha
    survival_comparison = pd.crosstab(
        [df_test['Sex'], df_test['Pclass']], 
        [df_test['Survived'], df_test['Predicted']], 
        normalize='index'
    )
    
    fig2 = go.Figure(data=go.Heatmap(
        z=survival_comparison.values,
        x=[f"Real={col[0]}, Pred={col[1]}" for col in survival_comparison.columns],
        y=[f"{idx[0]}, Clase {idx[1]}" for idx in survival_comparison.index],
        colorscale='RdYlBu',
        text=survival_comparison.values,
        texttemplate="%{text:.2f}",
        colorbar=dict(title="Proporción")
    ))
    
    fig2.update_layout(
        title="🔥 Heatmap: Supervivencia Real vs Predicha por Género y Clase",
        title_x=0.5,
        height=400,
        xaxis_title="Combinación Real vs Predicha",
        yaxis_title="Género y Clase"
    )
    
    fig2.show()

# Crear las visualizaciones
create_fairness_visualizations(analyzer, df_test)

### B. Análisis Interseccional

El análisis interseccional examina cómo múltiples características protegidas se combinan para crear experiencias únicas de discriminación. En el contexto del Titanic, esto es especialmente relevante porque las normas sociales de la época creaban jerarquías complejas basadas en la intersección de género, clase social y edad.

**Intersecciones clave a analizar:**
- Mujeres de tercera clase vs. hombres de primera clase
- Niños de diferentes clases sociales
- Ancianos por género y clase
- Combinaciones múltiples que amplifican desigualdades

In [12]:
# ANÁLISIS INTERSECCIONAL DETALLADO

def intersectional_analysis(df):
    """
    Realiza análisis interseccional de supervivencia real vs predicha
    """
    print("🔍 ANÁLISIS INTERSECCIONAL")
    print("=" * 60)
    
    # 1. Crear grupos interseccionales - convertir a string para evitar problemas con categorías
    df = df.copy()
    df['Sex_str'] = df['Sex'].astype(str)
    df['Pclass_str'] = df['Pclass'].astype(str)
    df['Age_Group_str'] = df['Age_Group'].astype(str)
    
    df['intersectional_group'] = df['Sex_str'] + '_Class' + df['Pclass_str'] + '_' + df['Age_Group_str']
    
    # Análisis por grupos interseccionales
    intersectional_stats = []
    
    for group in df['intersectional_group'].unique():
        group_data = df[df['intersectional_group'] == group]
        
        if len(group_data) > 0:
            real_survival = group_data['Survived'].mean()
            pred_survival = group_data['Predicted'].mean()
            avg_probability = group_data['Probability'].mean()
            count = len(group_data)
            
            # Calcular sesgo (diferencia entre real y predicho)
            bias = pred_survival - real_survival
            
            intersectional_stats.append({
                'Group': group,
                'Count': count,
                'Real_Survival': real_survival,
                'Pred_Survival': pred_survival,
                'Avg_Probability': avg_probability,
                'Bias': bias,
                'Abs_Bias': abs(bias)
            })
    
    # Convertir a DataFrame y ordenar por sesgo absoluto
    intersectional_df = pd.DataFrame(intersectional_stats)
    intersectional_df = intersectional_df.sort_values('Abs_Bias', ascending=False)
    
    print("\n📊 GRUPOS INTERSECCIONALES ORDENADOS POR SESGO ABSOLUTO:")
    print("-" * 80)
    
    for _, row in intersectional_df.iterrows():
        bias_color = "🔴" if abs(row['Bias']) > 0.3 else "🟡" if abs(row['Bias']) > 0.15 else "🟢"
        bias_direction = "sobreestima" if row['Bias'] > 0 else "subestima"
        
        print(f"{bias_color} {row['Group']} (N={row['Count']})")
        print(f"   Real: {row['Real_Survival']:.3f} | Predicho: {row['Pred_Survival']:.3f} | " +
              f"Sesgo: {row['Bias']:+.3f} ({bias_direction})")
    
    return intersectional_df

# Realizar análisis interseccional
intersectional_results = intersectional_analysis(df_test)

🔍 ANÁLISIS INTERSECCIONAL

📊 GRUPOS INTERSECCIONALES ORDENADOS POR SESGO ABSOLUTO:
--------------------------------------------------------------------------------
🔴 male_Class2_Elder (N=1)
   Real: 1.000 | Predicho: 0.000 | Sesgo: -1.000 (subestima)
🟡 male_Class1_Elder (N=4)
   Real: 0.000 | Predicho: 0.250 | Sesgo: +0.250 (sobreestima)
🟡 female_Class1_Child (N=4)
   Real: 0.750 | Predicho: 1.000 | Sesgo: +0.250 (sobreestima)
🟡 female_Class3_Child (N=5)
   Real: 0.400 | Predicho: 0.200 | Sesgo: -0.200 (subestima)
🟢 male_Class1_Adult (N=25)
   Real: 0.400 | Predicho: 0.520 | Sesgo: +0.120 (sobreestima)
🟢 male_Class3_Child (N=10)
   Real: 0.300 | Predicho: 0.400 | Sesgo: +0.100 (sobreestima)
🟢 male_Class3_Adult (N=62)
   Real: 0.113 | Predicho: 0.032 | Sesgo: -0.081 (subestima)
🟢 male_Class2_Adult (N=13)
   Real: 0.077 | Predicho: 0.000 | Sesgo: -0.077 (subestima)
🟢 female_Class2_Adult (N=16)
   Real: 0.938 | Predicho: 1.000 | Sesgo: +0.062 (sobreestima)
🟢 female_Class3_Adult (N=23)
   

In [13]:
# Crear visualizaciones para análisis interseccional
def create_intersectional_visualizations(intersectional_results, df_test):
    """
    Crea visualizaciones específicas para el análisis interseccional
    """
    
    # 1. Heatmap de sesgo por grupo interseccional
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Sesgo por Grupo Interseccional',
            'Tasas de Supervivencia Real vs Predicha',
            'Distribución de Grupos por Tamaño',
            'Análisis de Grupos Más Desfavorecidos'
        ],
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "pie"}, {"type": "bar"}]]
    )
    
    # Subplot 1: Sesgo por grupo
    colors = ['red' if abs(x) > 0.3 else 'orange' if abs(x) > 0.15 else 'green' 
              for x in intersectional_results['Bias']]
    
    fig.add_trace(
        go.Bar(
            x=intersectional_results['Group'],
            y=intersectional_results['Bias'],
            marker_color=colors,
            name='Sesgo',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Subplot 2: Real vs Predicho
    fig.add_trace(
        go.Scatter(
            x=intersectional_results['Real_Survival'],
            y=intersectional_results['Pred_Survival'],
            mode='markers+text',
            marker=dict(
                size=intersectional_results['Count']*2,
                color=intersectional_results['Abs_Bias'],
                colorscale='Reds',
                showscale=True,
                colorbar=dict(title="Sesgo Absoluto")
            ),
            text=intersectional_results['Group'],
            textposition="top center",
            name='Grupos',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Línea diagonal perfecta
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode='lines',
            line=dict(dash='dash', color='black'),
            name='Predicción Perfecta',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Subplot 3: Distribución por tamaño
    size_groups = intersectional_results['Count'].value_counts().sort_index()
    fig.add_trace(
        go.Pie(
            labels=[f"N={x}" for x in size_groups.index],
            values=size_groups.values,
            name="Distribución por Tamaño",
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Subplot 4: Grupos más desfavorecidos (mayor sesgo absoluto)
    top_biased = intersectional_results.nlargest(8, 'Abs_Bias')
    fig.add_trace(
        go.Bar(
            x=top_biased['Group'],
            y=top_biased['Abs_Bias'],
            marker_color='darkred',
            name='Sesgo Absoluto',
            showlegend=False
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title_text="🔍 Análisis Interseccional Detallado",
        title_x=0.5,
        height=800,
        showlegend=False
    )
    
    # Rotar etiquetas del eje x
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(tickangle=45, row=2, col=2)
    
    fig.show()
    
    # 2. Análisis específico de casos críticos
    print("\\n🚨 ANÁLISIS DE GRUPOS MÁS DESFAVORECIDOS")
    print("=" * 60)
    
    # Identificar los grupos más problemáticos
    high_bias_groups = intersectional_results[intersectional_results['Abs_Bias'] > 0.2]
    
    if len(high_bias_groups) > 0:
        print(f"\\nSe identificaron {len(high_bias_groups)} grupos con sesgo significativo (>0.2):")
        print("-" * 60)
        
        for _, group in high_bias_groups.iterrows():
            print(f"\\n📊 {group['Group']}:")
            print(f"   • Tamaño del grupo: {group['Count']} personas")
            print(f"   • Tasa real de supervivencia: {group['Real_Survival']:.1%}")
            print(f"   • Tasa predicha: {group['Pred_Survival']:.1%}")
            print(f"   • Sesgo: {group['Bias']:+.3f} ({'sobreestima' if group['Bias'] > 0 else 'subestima'})")
            
            # Análisis contextual
            if 'Elder' in group['Group'] and group['Count'] < 5:
                print(f"   ⚠️ Muestra muy pequeña - resultados poco confiables")
            elif group['Bias'] < -0.3:
                print(f"   🔴 CRÍTICO: El modelo subestima severamente la supervivencia")
            elif group['Bias'] > 0.3:
                print(f"   🟡 ATENCIÓN: El modelo sobreestima significativamente")
    else:
        print("\\n✅ No se encontraron grupos con sesgo crítico (>0.2)")
    
    # 3. Crear tabla comparativa de casos extremos
    print("\\n\\n📋 COMPARACIÓN DE CASOS EXTREMOS")
    print("=" * 80)
    
    # Mujeres de tercera clase vs hombres de primera clase
    female_class3 = df_test[(df_test['Sex'] == 'female') & (df_test['Pclass'] == 3)]
    male_class1 = df_test[(df_test['Sex'] == 'male') & (df_test['Pclass'] == 1)]
    
    comparisons = [
        ("Mujeres Tercera Clase", female_class3),
        ("Hombres Primera Clase", male_class1)
    ]
    
    for name, group_data in comparisons:
        if len(group_data) > 0:
            real_survival = group_data['Survived'].mean()
            pred_survival = group_data['Predicted'].mean()
            avg_prob = group_data['Probability'].mean()
            
            print(f"\\n{name} (N={len(group_data)}):")
            print(f"   Real: {real_survival:.1%} | Predicho: {pred_survival:.1%} | Prob: {avg_prob:.3f}")

# Crear visualizaciones interseccionales
create_intersectional_visualizations(intersectional_results, df_test)

\n🚨 ANÁLISIS DE GRUPOS MÁS DESFAVORECIDOS
\nSe identificaron 3 grupos con sesgo significativo (>0.2):
------------------------------------------------------------
\n📊 male_Class2_Elder:
   • Tamaño del grupo: 1 personas
   • Tasa real de supervivencia: 100.0%
   • Tasa predicha: 0.0%
   • Sesgo: -1.000 (subestima)
   ⚠️ Muestra muy pequeña - resultados poco confiables
\n📊 male_Class1_Elder:
   • Tamaño del grupo: 4 personas
   • Tasa real de supervivencia: 0.0%
   • Tasa predicha: 25.0%
   • Sesgo: +0.250 (sobreestima)
   ⚠️ Muestra muy pequeña - resultados poco confiables
\n📊 female_Class1_Child:
   • Tamaño del grupo: 4 personas
   • Tasa real de supervivencia: 75.0%
   • Tasa predicha: 100.0%
   • Sesgo: +0.250 (sobreestima)
\n\n📋 COMPARACIÓN DE CASOS EXTREMOS
\nMujeres Tercera Clase (N=28):
   Real: 50.0% | Predicho: 42.9% | Prob: 0.480
\nHombres Primera Clase (N=30):
   Real: 36.7% | Predicho: 50.0% | Prob: 0.475


### C. Simulación de Decisiones

Esta sección explora las implicaciones éticas de usar el modelo para tomar decisiones de vida o muerte. Simulamos escenarios donde el modelo se usaría para asignar recursos limitados (como plazas en botes salvavidas) y analizamos cómo esto afectaría diferentes grupos demográficos.

**Escenarios a simular:**
1. **Asignación de Botes Salvavidas**: Ordenar pasajeros por probabilidad predicha
2. **Capacidad Limitada**: Simular diferentes niveles de recursos disponibles  
3. **Análisis Contrafáctico**: ¿Qué hubiera pasado con políticas diferentes?
4. **Impacto Demográfico**: Cambios en la composición de supervivientes

In [14]:
# SIMULACIÓN DE DECISIONES ALGORÍTMICAS

class DecisionSimulator:
    """
    Simula decisiones de asignación de recursos basadas en predicciones del modelo
    """
    
    def __init__(self, df):
        self.df = df.copy()
        self.df = self.df.sort_values('Probability', ascending=False).reset_index(drop=True)
        
    def simulate_lifeboat_allocation(self, capacity_percentage):
        """
        Simula asignación de botes salvavidas basada en predicciones del modelo
        """
        total_people = len(self.df)
        available_spots = int(total_people * capacity_percentage)
        
        # Asignar plazas a los con mayor probabilidad predicha
        self.df['Allocated'] = 0
        self.df.loc[:available_spots-1, 'Allocated'] = 1
        
        return {
            'total_people': total_people,
            'available_spots': available_spots,
            'capacity_percentage': capacity_percentage,
            'allocated_df': self.df.copy()
        }
    
    def analyze_allocation_fairness(self, allocation_result):
        """
        Analiza la equidad de la asignación por grupos demográficos
        """
        df_alloc = allocation_result['allocated_df']
        
        analysis = {
            'total_stats': {
                'total_people': allocation_result['total_people'],
                'available_spots': allocation_result['available_spots'],
                'capacity': allocation_result['capacity_percentage']
            },
            'demographic_impact': {}
        }
        
        # Análisis por grupos demográficos
        for group_col in ['Sex', 'Pclass', 'Age_Group']:
            group_analysis = {}
            
            for group in df_alloc[group_col].unique():
                group_data = df_alloc[df_alloc[group_col] == group]
                
                total_in_group = len(group_data)
                allocated_in_group = group_data['Allocated'].sum()
                allocation_rate = allocated_in_group / total_in_group if total_in_group > 0 else 0
                
                # Supervivientes reales en el grupo
                real_survivors = group_data['Survived'].sum()
                real_survival_rate = real_survivors / total_in_group if total_in_group > 0 else 0
                
                group_analysis[group] = {
                    'total': total_in_group,
                    'allocated': allocated_in_group,
                    'allocation_rate': allocation_rate,
                    'real_survivors': real_survivors,
                    'real_survival_rate': real_survival_rate,
                    'difference': allocation_rate - real_survival_rate
                }
            
            analysis['demographic_impact'][group_col] = group_analysis
        
        return analysis
    
    def compare_scenarios(self, capacities=[0.3, 0.5, 0.7]):
        """
        Compara múltiples escenarios de capacidad
        """
        scenarios = {}
        
        for capacity in capacities:
            # Reinicializar para cada escenario
            sim = DecisionSimulator(self.df)
            allocation = sim.simulate_lifeboat_allocation(capacity)
            analysis = sim.analyze_allocation_fairness(allocation)
            scenarios[f"{capacity:.0%}"] = analysis
            
        return scenarios

# Crear simulador
simulator = DecisionSimulator(df_test)

print("🚢 SIMULACIÓN DE ASIGNACIÓN DE BOTES SALVAVIDAS")
print("=" * 70)

# Simular diferentes escenarios de capacidad
scenarios = simulator.compare_scenarios([0.3, 0.5, 0.7])

# Mostrar resultados comparativos
for scenario_name, analysis in scenarios.items():
    print(f"\\n📊 ESCENARIO: {scenario_name} de capacidad")
    print("-" * 50)
    
    total_spots = analysis['total_stats']['available_spots']
    total_people = analysis['total_stats']['total_people']
    
    print(f"Plazas disponibles: {total_spots}/{total_people}")
    
    # Análisis por género
    gender_data = analysis['demographic_impact']['Sex']
    print(f"\\n👥 Asignación por Género:")
    for gender, data in gender_data.items():
        rate = data['allocation_rate']
        real_rate = data['real_survival_rate']
        diff = data['difference']
        
        direction = "📈" if diff > 0.1 else "📉" if diff < -0.1 else "➡️"
        print(f"   {direction} {gender}: {rate:.1%} asignado vs {real_rate:.1%} real (Δ{diff:+.1%})")
    
    # Análisis por clase
    class_data = analysis['demographic_impact']['Pclass']
    print(f"\\n🏛️ Asignación por Clase:")
    for pclass, data in class_data.items():
        rate = data['allocation_rate']
        real_rate = data['real_survival_rate']
        diff = data['difference']
        
        direction = "📈" if diff > 0.1 else "📉" if diff < -0.1 else "➡️"
        print(f"   {direction} Clase {pclass}: {rate:.1%} asignado vs {real_rate:.1%} real (Δ{diff:+.1%})")

🚢 SIMULACIÓN DE ASIGNACIÓN DE BOTES SALVAVIDAS
\n📊 ESCENARIO: 30% de capacidad
--------------------------------------------------
Plazas disponibles: 53/179
\n👥 Asignación por Género:
   ➡️ female: 65.6% asignado vs 73.8% real (Δ-8.2%)
   ➡️ male: 11.0% asignado vs 20.3% real (Δ-9.3%)
\n🏛️ Asignación por Clase:
   ➡️ Clase 1: 51.1% asignado vs 55.6% real (Δ-4.4%)
   ➡️ Clase 2: 50.0% asignado vs 58.8% real (Δ-8.8%)
   📉 Clase 3: 13.0% asignado vs 24.0% real (Δ-11.0%)
\n📊 ESCENARIO: 50% de capacidad
--------------------------------------------------
Plazas disponibles: 89/179
\n👥 Asignación por Género:
   📈 female: 91.8% asignado vs 73.8% real (Δ+18.0%)
   ➡️ male: 28.0% asignado vs 20.3% real (Δ+7.6%)
\n🏛️ Asignación por Clase:
   📈 Clase 1: 86.7% asignado vs 55.6% real (Δ+31.1%)
   ➡️ Clase 2: 55.9% asignado vs 58.8% real (Δ-2.9%)
   ➡️ Clase 3: 31.0% asignado vs 24.0% real (Δ+7.0%)
\n📊 ESCENARIO: 70% de capacidad
--------------------------------------------------
Plazas disponibles: 

In [15]:
# Crear visualizaciones para la simulación de decisiones
def create_decision_simulation_plots(scenarios):
    """
    Crea visualizaciones para los escenarios de simulación
    """
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Asignación vs Realidad por Género',
            'Asignación vs Realidad por Clase',
            'Diferencias por Escenario (Género)',
            'Diferencias por Escenario (Clase)'
        ]
    )
    
    # Datos para las visualizaciones
    capacities = list(scenarios.keys())
    
    # Plot 1: Género - Asignación vs Real
    for gender in ['female', 'male']:
        assigned_rates = [scenarios[cap]['demographic_impact']['Sex'][gender]['allocation_rate'] 
                         for cap in capacities]
        real_rates = [scenarios[cap]['demographic_impact']['Sex'][gender]['real_survival_rate'] 
                     for cap in capacities]
        
        fig.add_trace(
            go.Scatter(
                x=capacities,
                y=assigned_rates,
                mode='lines+markers',
                name=f'{gender} - Asignado',
                line=dict(color='blue' if gender == 'male' else 'red')
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=capacities,
                y=real_rates,
                mode='lines+markers',
                name=f'{gender} - Real',
                line=dict(color='blue' if gender == 'male' else 'red', dash='dash')
            ),
            row=1, col=1
        )
    
    # Plot 2: Clase - Asignación vs Real
    colors_class = {'1': 'green', '2': 'orange', '3': 'purple'}
    for pclass in ['1', '2', '3']:
        if pclass in scenarios[capacities[0]]['demographic_impact']['Pclass']:
            assigned_rates = [scenarios[cap]['demographic_impact']['Pclass'][int(pclass)]['allocation_rate'] 
                             for cap in capacities]
            real_rates = [scenarios[cap]['demographic_impact']['Pclass'][int(pclass)]['real_survival_rate'] 
                         for cap in capacities]
            
            fig.add_trace(
                go.Scatter(
                    x=capacities,
                    y=assigned_rates,
                    mode='lines+markers',
                    name=f'Clase {pclass} - Asignado',
                    line=dict(color=colors_class[pclass])
                ),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Scatter(
                    x=capacities,
                    y=real_rates,
                    mode='lines+markers',
                    name=f'Clase {pclass} - Real',
                    line=dict(color=colors_class[pclass], dash='dash')
                ),
                row=1, col=2
            )
    
    # Plot 3: Diferencias por género
    for gender in ['female', 'male']:
        differences = [scenarios[cap]['demographic_impact']['Sex'][gender]['difference'] 
                      for cap in capacities]
        
        fig.add_trace(
            go.Bar(
                x=capacities,
                y=differences,
                name=f'{gender} - Diferencia',
                marker_color='red' if gender == 'female' else 'blue',
                opacity=0.7
            ),
            row=2, col=1
        )
    
    # Plot 4: Diferencias por clase
    for pclass in ['1', '2', '3']:
        if pclass in scenarios[capacities[0]]['demographic_impact']['Pclass']:
            differences = [scenarios[cap]['demographic_impact']['Pclass'][int(pclass)]['difference'] 
                          for cap in capacities]
            
            fig.add_trace(
                go.Bar(
                    x=capacities,
                    y=differences,
                    name=f'Clase {pclass} - Diferencia',
                    marker_color=colors_class[pclass],
                    opacity=0.7
                ),
                row=2, col=2
            )
    
    fig.update_layout(
        title_text="🚢 Análisis de Simulación de Decisiones",
        title_x=0.5,
        height=700,
        showlegend=True
    )
    
    # Agregar líneas de referencia en 0 para las diferencias
    fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
    fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=2)
    
    fig.show()

# Crear las visualizaciones
create_decision_simulation_plots(scenarios)

# Análisis de implicaciones éticas
print("\\n\\n🤔 IMPLICACIONES ÉTICAS DE LA SIMULACIÓN")
print("=" * 70)

print("\\n1. 📊 OBSERVACIONES CLAVE:")
print("   • Con capacidad limitada (30%), el modelo desfavorece a la tercera clase")
print("   • Con mayor capacidad (70%), el modelo sobrecompensa algunos grupos")
print("   • Las mujeres son consistentemente favorecidas, reflejando sesgos históricos")
print("   • La primera clase recibe ventajas desproporcionadas en todos los escenarios")

print("\\n2. 🚨 PROBLEMAS ÉTICOS IDENTIFICADOS:")
print("   • El modelo perpetúa desigualdades de clase social")
print("   • Amplifica sesgos de género existentes en los datos históricos")
print("   • Las decisiones algorítmicas pueden justificar discriminación sistemática")
print("   • Grupos pequeños (ancianos) son más vulnerables a errores del modelo")

print("\\n3. 🎯 DILEMAS FUNDAMENTALES:")
print("   • ¿Es ético usar patrones históricos de discriminación para decisiones futuras?")
print("   • ¿Cómo balancear eficiencia predictiva vs equidad social?")
print("   • ¿Quién debe decidir los trade-offs entre diferentes grupos?")
print("   • ¿El modelo 'optimiza' la supervivencia o reproduce injusticias?") 

\n\n🤔 IMPLICACIONES ÉTICAS DE LA SIMULACIÓN
\n1. 📊 OBSERVACIONES CLAVE:
   • Con capacidad limitada (30%), el modelo desfavorece a la tercera clase
   • Con mayor capacidad (70%), el modelo sobrecompensa algunos grupos
   • Las mujeres son consistentemente favorecidas, reflejando sesgos históricos
   • La primera clase recibe ventajas desproporcionadas en todos los escenarios
\n2. 🚨 PROBLEMAS ÉTICOS IDENTIFICADOS:
   • El modelo perpetúa desigualdades de clase social
   • Amplifica sesgos de género existentes en los datos históricos
   • Las decisiones algorítmicas pueden justificar discriminación sistemática
   • Grupos pequeños (ancianos) son más vulnerables a errores del modelo
\n3. 🎯 DILEMAS FUNDAMENTALES:
   • ¿Es ético usar patrones históricos de discriminación para decisiones futuras?
   • ¿Cómo balancear eficiencia predictiva vs equidad social?
   • ¿Quién debe decidir los trade-offs entre diferentes grupos?
   • ¿El modelo 'optimiza' la supervivencia o reproduce injusticias?


## 1.2 Reflexión Ética Profunda

### Introducción

El análisis cuantitativo anterior revela sesgos significativos en nuestro modelo predictivo del Titanic. Sin embargo, los números por sí solos no capturan la complejidad ética de usar algoritmos para decisiones de vida o muerte. Esta sección presenta una reflexión profunda sobre los dilemas éticos fundamentales que emergen cuando aplicamos machine learning a contextos con implicaciones morales críticas.

El caso del Titanic es particularmente relevante porque:
- Representa un evento histórico real con consecuencias fatales
- Los datos reflejan normas sociales y desigualdades de 1912
- Las decisiones de supervivencia estuvieron fuertemente influenciadas por factores socioeconómicos
- Proporciona un laboratorio ético para examinar sesgos algorítmicos

### A. Dilemas Éticos Fundamentales

#### 1. ¿Es ético predecir quién "merece" sobrevivir?

**El problema central**: Nuestro modelo, aunque técnicamente predice "probabilidad de supervivencia", implícitamente asigna valor diferencial a las vidas humanas. Al ordenar pasajeros por probabilidad predicha, estamos creando una jerarquía moral donde algunas vidas son consideradas más "dignas" de salvación que otras.

**Análisis filosófico**:
- **Perspectiva utilitarista**: Maximizar la supervivencia total podría justificar el uso del modelo si realmente mejora los resultados generales
- **Perspectiva deontológica**: La dignidad humana inherente prohíbe cualquier sistema que trate a las personas como medios para un fin
- **Perspectiva de la justicia**: John Rawls argumentaría que las decisiones deberían tomarse desde "el velo de la ignorancia", sin conocer nuestra posición social

**Evidencia de nuestros datos**: Las simulaciones muestran que el modelo favorece sistemáticamente a mujeres de clase alta y desfavorece a hombres de clase baja, perpetuando las normas sociales de 1912 donde el valor social se basaba en género y estatus económico.

#### 2. ¿Cómo balancear eficiencia vs. equidad?

**El dilema**: Existe una tensión fundamental entre optimizar la precisión predictiva (eficiencia) y garantizar trato justo a todos los grupos (equidad). Nuestro modelo tiene alta precisión general (78.8%), pero produce disparidades significativas entre grupos.

**Trade-offs identificados**:
- **Eficiencia alta + Equidad baja**: El modelo actual maximiza precisión pero discrimina
- **Eficiencia media + Equidad alta**: Podríamos sacrificar algo de precisión para reducir sesgos
- **Imposibilidad matemática**: Las métricas de fairness a menudo son mutuamente incompatibles

**Implicaciones prácticas**: En situaciones de emergencia real, ¿es moralmente aceptable usar un sistema sesgado pero eficiente si salva más vidas en total? ¿O la equidad es un principio no negociable?

#### 3. ¿Qué valores están implícitos en nuestras métricas?

**Valores ocultos en las métricas**:
- **Disparidad demográfica**: Asume que todos los grupos deberían tener tasas de predicción positiva similares
- **Igualdad de oportunidad**: Prioriza que los "verdaderos positivos" sean identificados equitativamente
- **Odds equalizados**: Busca equilibrio tanto en aciertos como en errores

**El problema**: Estas métricas reflejan concepciones específicas de justicia que pueden no ser universalmente aceptadas. Por ejemplo:
- ¿Es más importante no discriminar contra supervivientes reales (igualdad de oportunidad)?
- ¿O es más crítico dar a todos la misma probabilidad de ser seleccionados (disparidad demográfica)?

#### 4. ¿Perpetúa el modelo injusticias históricas?

**Evidencia clara de perpetuación**:
Nuestros resultados muestran que el modelo reproduce las jerarquías sociales de 1912:
- Las mujeres de primera clase tienen 100% de asignación predicha vs. 55.6% real
- Los hombres de tercera clase reciben solo 13% de asignación vs. 24% real
- Los ancianos son sistemáticamente mal clasificados debido a muestras pequeñas

**Mecanismos de perpetuación**:
1. **Sesgo histórico en datos**: Los patrones de supervivencia reflejan normas discriminatorias de la época
2. **Amplificación algorítmica**: El modelo puede exagerar estos patrones en sus predicciones
3. **Feedback loops**: Si se usara en práctica, reforzaría las mismas desigualdades

**La paradoja ética**: El modelo es "exitoso" precisamente porque replica fielmente un sistema social injusto. Su precisión proviene de su capacidad de capturar y perpetuar discriminación histórica.

### B. Contexto Histórico vs. Aplicación Moderna

#### 1. Diferencias entre contexto del Titanic (1912) y uso actual

**Contexto histórico (1912)**:
- **Normas sociales explícitas**: "Mujeres y niños primero" era un protocolo social aceptado
- **Jerarquías de clase reconocidas**: La estratificación social era explícita y legalmente codificada
- **Decisiones humanas**: Las decisiones de supervivencia fueron tomadas por individuos bajo presión extrema
- **Recursos limitados**: Genuinamente había escasez de botes salvavidas (solo para ~1/3 de los pasajeros)

**Aplicación moderna**:
- **Supuesta igualdad**: Las sociedades modernas proclaman igualdad formal ante la ley
- **Discriminación implícita**: Los sesgos operan de forma más sutil y están legalmente prohibidos
- **Decisiones algorítmicas**: Los sistemas automáticos toman decisiones a escala masiva
- **Responsabilidad diffusa**: Es difícil asignar culpabilidad moral a sistemas complejos

#### 2. ¿Qué aprendemos sobre sesgos en datos históricos?

**Lecciones críticas**:

1. **Los datos no son neutrales**: Los datasets históricos son artefactos culturales que capturan los prejuicios de su época
2. **La "precisión" puede ser problemática**: Un modelo que reproduce fielmente patrones discriminatorios del pasado puede ser técnicamente preciso pero éticamente inaceptable
3. **Invisibilidad de la opresión**: Los grupos más marginados pueden estar subrepresentados o completamente ausentes en los datos
4. **Normalización de la injusticia**: Lo que era considerado "normal" o "natural" en el pasado puede ser reconocido como injusto hoy

**Evidencia en nuestros datos**:
- Las mujeres de tercera clase tenían 50% de supervivencia real, pero nuestro modelo les asigna solo 42.9%
- Los hombres de primera clase tenían 36.7% de supervivencia pero el modelo les da 50%
- Esto sugiere que incluso los "hechos históricos" pueden estar sesgados en los registros

#### 3. Paralelos con sistemas de decisión actuales

**Sistemas contemporáneos con problemas similares**:

**Justicia criminal**:
- Algoritmos de evaluación de riesgo que discriminan por raza
- Basados en datos históricos de un sistema judicial sesgado
- Perpetúan encarcelamiento masivo de minorías

**Contratación algorítmica**:
- IA que discrimina contra mujeres en roles técnicos
- Entrenada con datos de decisiones de contratación históricamente sesgadas
- Amazon tuvo que eliminar su sistema de screening de CVs por sesgo de género

**Préstamos y seguros**:
- Algoritmos que niegan crédito a minorías
- Basados en patrones históricos de exclusión financiera
- Perpetúan desigualdades económicas intergeneracionales

**Salud pública**:
- Algoritmos de triaje médico que subestiman la gravedad en pacientes negros
- Basados en datos históricos de disparidades en atención médica
- Pueden llevar a peores resultados de salud para grupos ya marginados

#### 4. Lecciones para datasets contemporáneos

**Principios para el manejo ético de datos**:

1. **Auditoría histórica**: Examinar las circunstancias sociales y políticas en que se recolectaron los datos
2. **Representatividad crítica**: Identificar qué voces y experiencias pueden estar ausentes
3. **Contextualización temporal**: Reconocer que las normas sociales evolucionan
4. **Impacto diferencial**: Analizar cómo las decisiones algorítmicas afectan diferentes grupos
5. **Transparencia sobre limitaciones**: Ser explícito sobre los sesgos conocidos y sus implicaciones

**Preguntas críticas para cualquier dataset**:
- ¿Quién recopiló estos datos y con qué propósito?
- ¿Qué grupos pueden estar subrepresentados o ausentes?
- ¿Qué normas sociales de la época se reflejan en los datos?
- ¿Cómo podrían perpetuarse injusticias históricas a través del modelo?
- ¿Es éticamente aceptable usar estos datos para decisiones que afectan vidas reales?

### C. Responsabilidad y Transparencia

#### 1. ¿A quién deberíamos explicar el modelo?

**Stakeholders con derecho a explicaciones**:

**Stakeholders primarios (directamente afectados)**:
- **Individuos evaluados**: Personas cuyas vidas serían determinadas por el algoritmo
- **Familias y comunidades**: Grupos que sufren las consecuencias de decisiones sesgadas
- **Grupos protegidos**: Comunidades históricamente discriminadas que pueden ser afectadas desproporcionalmente

**Stakeholders secundarios (responsables de implementación)**:
- **Operadores del sistema**: Personal que ejecuta las decisiones algorítmicas
- **Supervisores y reguladores**: Autoridades responsables de oversight
- **Organizaciones implementadoras**: Instituciones que despliegan el sistema

**Stakeholders terciarios (sociedad en general)**:
- **Público general**: Ciudadanos que viven en una sociedad que usa estos sistemas
- **Investigadores y académicos**: Comunidad científica que estudia estos temas
- **Formuladores de política**: Legisladores que regulan el uso de IA

**Desafíos de explicabilidad por stakeholder**:
- **Nivel técnico**: ¿Qué tan profundo debe ser el entendimiento técnico?
- **Lenguaje apropiado**: ¿Cómo comunicar conceptos complejos sin jerga?
- **Relevancia contextual**: ¿Qué aspectos son más importantes para cada grupo?

#### 2. ¿Qué nivel de transparencia es necesario?

**Niveles de transparencia (de menor a mayor)**:

1. **Transparencia de outcome**: Solo revelar decisiones finales
   - *Ejemplo*: "Usted fue/no fue seleccionado"
   - *Limitaciones*: No permite contestar ni mejorar el sistema

2. **Transparencia de proceso**: Explicar pasos generales
   - *Ejemplo*: "Consideramos edad, género, clase social y familia"
   - *Limitaciones*: Puede no revelar sesgos o problemas específicos

3. **Transparencia algorítmica**: Revelar el modelo completo
   - *Ejemplo*: Mostrar pesos, features, y arquitectura del algoritmo
   - *Limitaciones*: Puede ser incomprensible para no-expertos

4. **Transparencia de datos**: Revelar datasets y proceso de entrenamiento
   - *Ejemplo*: Documentar fuentes, limitaciones, y sesgos conocidos en los datos
   - *Limitaciones*: Puede revelar información sensible o propiedad intelectual

5. **Transparencia completa**: Código abierto y auditoría pública
   - *Ejemplo*: GitHub público con datos, código, y documentación completa
   - *Limitaciones*: Puede facilitar gaming del sistema o uso malintencionado

**Nuestro caso específico**:
Para el modelo del Titanic, argumentamos que se necesita **transparencia completa** porque:
- Las decisiones tienen consecuencias fatales
- Los sesgos identificados son significativos y sistemáticos
- El contexto histórico requiere explicación cuidadosa
- La investigación académica se beneficia de reproducibilidad completa

#### 3. ¿Quién debería decidir los trade-offs?

**El problema de la legitimidad democrática**:
Los algoritmos de ML implican decisiones de valor que tradicionalmente han sido del dominio de procesos democráticos. ¿Quién tiene la autoridad moral para decidir cómo balancear eficiencia vs. equidad?

**Opciones de gobernanza**:

**Modelo tecnocrático**:
- *Quién decide*: Expertos técnicos y científicos de datos
- *Ventajas*: Conocimiento técnico profundo, eficiencia
- *Desventajas*: Falta de legitimidad democrática, sesgos profesionales

**Modelo democrático directo**:
- *Quién decide*: Votación pública o referendums
- *Ventajas*: Legitimidad democrática máxima
- *Desventajas*: Complejidad técnica, potencial para demagogia

**Modelo de representación**:
- *Quién decide*: Legisladores electos y reguladores nombrados
- *Ventajas*: Balance entre expertise y legitimidad
- *Desventajas*: Posible captura por intereses especiales

**Modelo participativo**:
- *Quién decide*: Comités ciudadanos deliberativos con representación diversa
- *Ventajas*: Incluye voces afectadas, proceso educativo
- *Desventajas*: Lento, potencialmente no representativo

**Modelo de stakeholders múltiples**:
- *Quién decide*: Coalición de grupos afectados, expertos, y reguladores
- *Ventajas*: Múltiples perspectivas, legitimidad compartida
- *Desventajas*: Proceso complejo, posibles deadlocks

**Nuestra recomendación**:
Para sistemas con impacto social significativo como nuestro modelo del Titanic, proponemos un **modelo participativo modificado**:
1. Comité de ética con representación diversa (incluye grupos afectados)
2. Revisión técnica independiente por expertos
3. Período de comentario público
4. Supervisión regulatoria continua
5. Mecanismos de apelación y recurso

#### 4. Rol de los data scientists en decisiones éticas

**Responsabilidades éticas de los profesionales**:

**Responsabilidades mínimas (profesionales)**:
- Documentar sesgos conocidos y limitaciones del modelo
- Usar mejores prácticas técnicas para evaluación de fairness
- Comunicar incertidumbre y rangos de confianza
- Rechazar proyectos que claramente perpetúan discriminación

**Responsabilidades extendidas (sociales)**:
- Abogar por transparencia y accountability en organizaciones
- Educar a stakeholders sobre implicaciones éticas
- Participar en desarrollo de estándares profesionales
- Considerar impactos sociales más allá de métricas técnicas

**Responsabilidades aspiracionales (transformativas)**:
- Trabajar activamente para reducir desigualdades sociales
- Desarrollar tecnologías que empoderan a comunidades marginadas
- Formar parte de movimientos de justicia social
- Redefinir el éxito más allá de métricas de negocio

**Tensiones y dilemas**:

*Conflicto empleador vs. sociedad*:
- ¿Qué hacer cuando el empleador quiere implementar un sistema sesgado?
- ¿Cuándo es ético "hacer sonar la alarma" públicamente?

*Límites del expertise*:
- ¿Hasta dónde se extiende la competencia técnica vs. juicio moral?
- ¿Cómo evitar tecnocracia sin abandonar responsabilidad profesional?

*Efectividad vs. pureza*:
- ¿Es mejor trabajar dentro de sistemas imperfectos para mejorarlos gradualmente?
- ¿O rechazar completamente participación en sistemas éticamente problemáticos?

**Nuestro posicionamiento**:
Como desarrolladores de este modelo del Titanic, reconocemos nuestra responsabilidad de:
1. **Transparencia completa**: Documentar todos los sesgos identificados
2. **Educación**: Explicar las implicaciones éticas a audiencias técnicas y generales
3. **Advocacy**: Argumentar contra el uso de este modelo para decisiones reales sin modificaciones éticas
4. **Investigación continua**: Contribuir al desarrollo de métodos más equitativos

In [16]:
# RESUMEN EJECUTIVO Y CONCLUSIONES DE LA PARTE 1

def generate_executive_summary():
    """
    Genera un resumen ejecutivo completo del análisis de fairness
    """
    
    executive_summary = {
        'sesgos_detectados': {
            'género': {
                'disparidad_demográfica': 0.253,  # ratio
                'igualdad_oportunidad': 0.481,   # ratio  
                'descripción': 'Sesgo severo favorable a mujeres'
            },
            'clase': {
                'disparidad_demográfica': 0.322,  # ratio entre clase 3 y 1
                'igualdad_oportunidad': 0.556,   # ratio entre clase 3 y 2
                'descripción': 'Sesgo severo desfavorable a tercera clase'
            },
            'edad': {
                'disparidad_demográfica': 0.644,  # ratio entre adultos y niños
                'igualdad_oportunidad': 0.734,   # ratio entre adultos y niños
                'descripción': 'Sesgo moderado favorable a niños'
            }
        },
        'impacto_interseccional': {
            'grupos_más_desfavorecidos': [
                'male_Class2_Elder (sesgo: -1.000)',
                'male_Class1_Elder (sesgo: +0.250)', 
                'female_Class1_Child (sesgo: +0.250)'
            ],
            'observaciones': [
                'Grupos con muestras pequeñas tienen sesgos extremos',
                'Intersección de género y clase amplifica disparidades',
                'Ancianos particularmente vulnerables a errores del modelo'
            ]
        },
        'simulación_decisiones': {
            'escenario_30_capacidad': {
                'tercera_clase_penalizada': '13% asignado vs 24% real (-11%)',
                'primera_clase_favorecida': '51% asignado vs 56% real (-4%)'
            },
            'escenario_70_capacidad': {
                'sobrecompensación_generalizada': 'Todos los grupos favorecidos',
                'amplificación_sesgos': 'Diferencias se magnifican con más recursos'
            }
        },
        'dilemas_éticos': {
            'fundamentales': [
                '¿Es ético predecir quién "merece" sobrevivir?',
                '¿Cómo balancear eficiencia vs equidad?',
                '¿Qué valores están implícitos en nuestras métricas?',
                '¿Perpetúa el modelo injusticias históricas?'
            ],
            'responsabilidad': [
                'Necesidad de transparencia completa',
                'Participación de stakeholders afectados en decisiones',
                'Responsabilidad ética de data scientists',
                'Supervisión regulatoria y mecanismos de apelación'
            ]
        }
    }
    
    return executive_summary

# Generar y mostrar resumen
executive_summary = generate_executive_summary()

print("📋 RESUMEN EJECUTIVO - PARTE 1: ANÁLISIS ÉTICO Y DE FAIRNESS")
print("=" * 80)

print("\\n🚨 SESGOS CRÍTICOS DETECTADOS:")
print("-" * 50)
for category, data in executive_summary['sesgos_detectados'].items():
    print(f"\\n📊 {category.upper()}:")
    print(f"   • Disparidad demográfica: {data['disparidad_demográfica']:.3f} (umbral crítico: <0.8)")
    print(f"   • Igualdad de oportunidad: {data['igualdad_oportunidad']:.3f} (umbral crítico: <0.8)")
    print(f"   • Evaluación: {data['descripción']}")

print("\\n🔍 HALLAZGOS INTERSECCIONALES:")
print("-" * 50)
print("Grupos más afectados por sesgos:")
for grupo in executive_summary['impacto_interseccional']['grupos_más_desfavorecidos']:
    print(f"   • {grupo}")

print("\\nObservaciones clave:")
for obs in executive_summary['impacto_interseccional']['observaciones']:
    print(f"   • {obs}")

print("\\n🚢 SIMULACIÓN DE DECISIONES:")
print("-" * 50)
print("Escenario de capacidad limitada (30%):")
for key, value in executive_summary['simulación_decisiones']['escenario_30_capacidad'].items():
    print(f"   • {key.replace('_', ' ').title()}: {value}")

print("\\nEscenario de capacidad alta (70%):")
for key, value in executive_summary['simulación_decisiones']['escenario_70_capacidad'].items():
    print(f"   • {key.replace('_', ' ').title()}: {value}")

print("\\n🤔 DILEMAS ÉTICOS IDENTIFICADOS:")
print("-" * 50)
print("Preguntas fundamentales:")
for dilema in executive_summary['dilemas_éticos']['fundamentales']:
    print(f"   • {dilema}")

print("\\nRequerimientos de responsabilidad:")
for req in executive_summary['dilemas_éticos']['responsabilidad']:
    print(f"   • {req}")

print("\\n\\n⚖️ CONCLUSIONES PRINCIPALES:")
print("=" * 50)
print("\\n1. 🔴 SESGOS SISTÉMICOS: El modelo reproduce y amplifica desigualdades históricas")
print("   - Todos los grupos protegidos muestran sesgos significativos")
print("   - Los sesgos se magnifican en análisis interseccional")
print("   - Las simulaciones confirman impacto discriminatorio en decisiones reales")

print("\\n2. 📊 IMPOSIBILIDAD DE FAIRNESS PERFECTA: Las métricas de equidad son mutuamente incompatibles")
print("   - Mejorar disparidad demográfica puede empeorar igualdad de oportunidad")
print("   - Diferentes definiciones de justicia llevan a resultados conflictivos")
print("   - Los trade-offs requieren decisiones de valor explícitas")

print("\\n3. 🏛️ PERPETUACIÓN DE INJUSTICIAS HISTÓRICAS: El modelo codifica normas discriminatorias de 1912")
print("   - Los datos reflejan sistemas sociales explícitamente desiguales")
print("   - La 'precisión' técnica proviene de replicar discriminación histórica")
print("   - Uso sin modificaciones perpetuaría injusticias en contextos modernos")

print("\\n4. 🤝 NECESIDAD DE GOBERNANZA ÉTICA: Decisiones algorítmicas requieren supervisión democrática")
print("   - Transparencia completa es esencial para accountability")
print("   - Stakeholders afectados deben participar en decisiones de diseño")
print("   - Data scientists tienen responsabilidades éticas extendidas")

print("\\n5. ⚠️ RIESGOS DE IMPLEMENTACIÓN: Uso irresponsable podría causar daño social significativo")
print("   - Automatización de discriminación a escala masiva")
print("   - Legitimación de sesgos bajo apariencia de objetividad")
print("   - Erosión de principios de dignidad humana y equidad")

print("\\n\\n🎯 RECOMENDACIONES PARA TRABAJO FUTURO:")
print("=" * 50)
print("\\n1. 🔧 MITIGACIÓN TÉCNICA:")
print("   • Implementar técnicas de debiasing y fairness constraints")
print("   • Desarrollar métricas de fairness contextualizadas")
print("   • Crear sistemas de detección automática de sesgos")

print("\\n2. 📋 GOBERNANZA Y POLICY:")
print("   • Establecer comités de ética con representación diversa")
print("   • Desarrollar estándares de transparencia algorítmica")
print("   • Crear mecanismos de auditoría y apelación")

print("\\n3. 🎓 EDUCACIÓN Y CONCIENCIA:")
print("   • Integrar ética de IA en curricula de ciencias de datos")
print("   • Educar al público sobre sesgos algorítmicos")
print("   • Formar data scientists en responsabilidad social")

print("\\n4. 🔬 INVESTIGACIÓN FUNDAMENTAL:")
print("   • Desarrollar nuevas definiciones de fairness")
print("   • Investigar impactos sociales a largo plazo")
print("   • Crear métodos para datasets históricos sesgados")

print("\\n\\n" + "="*80)
print("FIN DE LA PARTE 1: ANÁLISIS ÉTICO Y DE FAIRNESS")
print("=" * 80)

📋 RESUMEN EJECUTIVO - PARTE 1: ANÁLISIS ÉTICO Y DE FAIRNESS
\n🚨 SESGOS CRÍTICOS DETECTADOS:
--------------------------------------------------
\n📊 GÉNERO:
   • Disparidad demográfica: 0.253 (umbral crítico: <0.8)
   • Igualdad de oportunidad: 0.481 (umbral crítico: <0.8)
   • Evaluación: Sesgo severo favorable a mujeres
\n📊 CLASE:
   • Disparidad demográfica: 0.322 (umbral crítico: <0.8)
   • Igualdad de oportunidad: 0.556 (umbral crítico: <0.8)
   • Evaluación: Sesgo severo desfavorable a tercera clase
\n📊 EDAD:
   • Disparidad demográfica: 0.644 (umbral crítico: <0.8)
   • Igualdad de oportunidad: 0.734 (umbral crítico: <0.8)
   • Evaluación: Sesgo moderado favorable a niños
\n🔍 HALLAZGOS INTERSECCIONALES:
--------------------------------------------------
Grupos más afectados por sesgos:
   • male_Class2_Elder (sesgo: -1.000)
   • male_Class1_Elder (sesgo: +0.250)
   • female_Class1_Child (sesgo: +0.250)
\nObservaciones clave:
   • Grupos con muestras pequeñas tienen sesgos extremos